In [1]:
%run util.py

In [2]:
data_folder = '/home/jzh/Waze/'

In [3]:
import json

In [4]:
with open(data_folder + 'jam_ref_data.json', 'r') as json_file:
    jam_ref_data = json.load(json_file)

with open(data_folder + 'jam_test_data.json', 'r') as json_file:
    jam_test_data = json.load(json_file)

In [5]:
# number of jams in reference data
print('number of jams in ref data: \n')
print(len(jam_ref_data))

# number of jams in test data
print('number of jams in test data: \n')
print(len(jam_test_data))

number of jams in ref data: 

3218
number of jams in test data: 

3882


In [6]:
[distToClusterCenter_delay, loc_label_delay, loc_centroid_delay] = zload(data_folder + 'loc_quant_results_delay.pkz')
[time_range_delay, KL_list_delay, etaWC_list_delay, win_jam_delay] = zload(data_folder + 'detec_results_delay.pkz')
para_dict_delay = zload(data_folder + 'para_dict_delay.pkz')

In [7]:
loc_centroid_delay

[[-71.085217,
  42.344827,
  -71.085217,
  42.347836,
  -71.0831385,
  42.344827,
  -71.0831385,
  42.347836],
 [-71.111864,
  42.364303,
  -71.111864,
  42.366344,
  -71.105036,
  42.364303,
  -71.105036,
  42.366344],
 [-71.110251,
  42.335729,
  -71.110251,
  42.338452,
  -71.106477,
  42.335729,
  -71.106477,
  42.338452]]

In [8]:
latitudes_delay = []

for i in range(3):
    for j in range(4):
        latitudes_delay.append(loc_centroid_delay[i][2*j+1])

In [9]:
longitudes_delay = []

for i in range(3):
    for j in range(4):
        longitudes_delay.append(loc_centroid_delay[i][2*j])

In [10]:
import gmplot

gmap = gmplot.GoogleMapPlotter(42.351848, -71.110730, 13)

gmap.scatter([42.351848], [-71.110730], c='g', marker=True)

gmap.circle(42.351848, -71.110730, 3000, '#ffefef', ew=4)

gmap.scatter(latitudes_delay, longitudes_delay, c='r', marker=True)

gmap.draw('./loc_clusters_delay.html')

In [11]:
len(time_range_delay), len(KL_list_delay), len(etaWC_list_delay), time_range_delay[0:2]

(286, 286, 286, [0.0, 5.0])

In [12]:
win_size = para_dict_delay['win_size']

alarmWinIdxList = []
alarmWinIdxStartTimeDict = {}
potential_ano_list = []
for i in range(len(KL_list_delay)):
    if KL_list_delay[i] > etaWC_list_delay[i]:
        alarmWinIdxList.append(i)
        alarmWinIdxStartTimeDict[i] = 60 * time_range_delay[i] + win_size
        for j in range(len(win_jam_delay[str(i)])):
            potential_ano_list.append(win_jam_delay[str(i)][j])

In [13]:
len(potential_ano_list)

910

In [14]:
alarmWinIdxStartTimeDict, alarmWinIdxList

({156: 47400.0,
  157: 47700.0,
  158: 48000.0,
  159: 48300.0,
  186: 56400.0,
  187: 56700.0,
  232: 70200.0,
  233: 70500.0,
  252: 76200.0,
  253: 76500.0,
  258: 78000.0,
  259: 78300.0,
  260: 78600.0,
  261: 78900.0},
 [156, 157, 158, 159, 186, 187, 232, 233, 252, 253, 258, 259, 260, 261])

In [15]:
with open(data_folder + 'sample_path_jam_test_data_sorted_delay.json', 'r') as json_file:
    sample_path_jam_test_data_sorted = json.load(json_file)
    
with open(data_folder + 'sample_path_jam_test_data_sorted_trans_delay.json', 'r') as json_file:
    sample_path_jam_test_data_sorted_trans = json.load(json_file)
    
with open(data_folder + 'sample_path_jam_ref_data_sorted_delay.json', 'r') as json_file:
    sample_path_jam_ref_data_sorted = json.load(json_file)
    
with open(data_folder + 'sample_path_jam_ref_data_sorted_trans_delay.json', 'r') as json_file:
    sample_path_jam_ref_data_sorted_trans = json.load(json_file)

In [16]:
delayTestList = []
for i in range(len(sample_path_jam_ref_data_sorted)):
    delayTestList.append(sample_path_jam_ref_data_sorted[str(i)]['delay'])

In [17]:
mu_delay = np.mean(delayTestList)
sigma_delay = np.std(delayTestList)

# apply the three-sigma rule
thres_delay = mu_delay + 3 * sigma_delay

In [18]:
thres_delay

504.49397326659039

In [19]:
# len(set(potential_ano_list)), set(potential_ano_list)

In [20]:
len(sample_path_jam_test_data_sorted_trans)

3881

In [21]:
# Refine the detection results; if a jam's length is larger than thres_length,
# then the jam is "indeed" abnormal.
ano_list = []
for key in set(potential_ano_list):
    if (sample_path_jam_test_data_sorted[key]['delay'] > thres_delay):
        ano_list.append(key)

In [22]:
len(ano_list), ano_list

(4, ['3094', '1502', '3126', '2412'])

In [23]:
ano_dict = {'idx_' + str(i):int(ano_list[i]) for i in range(len(ano_list))}

In [24]:
ano_dict

{'idx_0': 3094, 'idx_1': 1502, 'idx_2': 3126, 'idx_3': 2412}

In [25]:
import scipy.io

scipy.io.savemat('./anoDict_delay.mat', ano_dict)

In [26]:
anoStartTimeDict = {}
for key in ano_list:
    anoStartTimeDict[key] = sample_path_jam_test_data_sorted[key]['startTime'] / 60.0

In [27]:
anoStartTimeDict

{'1502': 935.0012733333333,
 '2412': 1165.0060833333334,
 '3094': 1295.0044566666666,
 '3126': 1295.0054400000001}

In [28]:
anoFirstAlarmTimeDict = {}
for key in ano_list:
    for winIdx in alarmWinIdxList:
        if key in win_jam_delay[str(winIdx)]:
            anoFirstAlarmTimeDict[key] = alarmWinIdxStartTimeDict[winIdx] / 60.0
            break

In [29]:
sample_path_jam_test_data_sorted['739']['startTime'], sample_path_jam_test_data_sorted['792']['startTime']

(44700.0239, 44700.3521)

In [30]:
anoFirstAlarmTimeDict

{'1502': 940.0, '2412': 1170.0, '3094': 1300.0, '3126': 1300.0}

In [31]:
alarmWinIdxStartTimeDict

{156: 47400.0,
 157: 47700.0,
 158: 48000.0,
 159: 48300.0,
 186: 56400.0,
 187: 56700.0,
 232: 70200.0,
 233: 70500.0,
 252: 76200.0,
 253: 76500.0,
 258: 78000.0,
 259: 78300.0,
 260: 78600.0,
 261: 78900.0}

In [32]:
for key in ano_list:
    print(sample_path_jam_test_data_sorted[key])

{u'uuid': u'182a7efa-737b-39e6-a532-bfb8c3288ca8', u'numPts': 9.0, u'delay': 585.0, u'length': 844.0, u'startTime': 77700.2674, u'alpha': 3.0, u'speed': 1.1944444444444444}
{u'uuid': u'a4e0d021-ad2d-3b6d-a77d-d3290e8f98f5', u'numPts': 26.0, u'delay': 643.0, u'length': 3568.0, u'startTime': 56100.0764, u'alpha': 1.0, u'speed': 4.388888888888889}
{u'uuid': u'eabd1821-56d8-3875-9d26-54e4e0857e37', u'numPts': 15.0, u'delay': 509.0, u'length': 897.0, u'startTime': 77700.3264, u'alpha': 3.0, u'speed': 1.5638888888888889}
{u'uuid': u'f2442303-7548-31e5-b1d1-1f2624a6d93a', u'numPts': 7.0, u'delay': 528.0, u'length': 1293.0, u'startTime': 69900.365, u'alpha': 5.0, u'speed': 1.9555555555555555}


### run the following cells after obtaining matched alerts in the 11th notebook

In [33]:
with open('./filteredAlerts.json', 'r') as json_file:
    filteredAlerts = json.load(json_file)

In [34]:
filteredAlerts

{u'idx_2950': {u'alertType': u'JAM',
  u'endTime': 75960.0,
  u'latitude': u'42.359444',
  u'longitude': u'-71.120531',
  u'startTime': 75900.0},
 u'idx_2951': {u'alertType': u'JAM',
  u'endTime': 75960.0,
  u'latitude': u'42.360081',
  u'longitude': u'-71.119965',
  u'startTime': 75900.0},
 u'idx_3030': {u'alertType': u'JAM',
  u'endTime': 79560.0,
  u'latitude': u'42.34357',
  u'longitude': u'-71.087383',
  u'startTime': 79500.0},
 u'idx_3031': {u'alertType': u'JAM',
  u'endTime': 79560.0,
  u'latitude': u'42.37798',
  u'longitude': u'-71.107817',
  u'startTime': 79500.0},
 u'idx_3032': {u'alertType': u'JAM',
  u'endTime': 79560.0,
  u'latitude': u'42.371509',
  u'longitude': u'-71.088811',
  u'startTime': 79500.0},
 u'idx_3033': {u'alertType': u'WEATHERHAZARD',
  u'endTime': 79560.0,
  u'latitude': u'42.331925',
  u'longitude': u'-71.117927',
  u'startTime': 79500.0},
 u'idx_3034': {u'alertType': u'JAM',
  u'endTime': 79560.0,
  u'latitude': u'42.354571',
  u'longitude': u'-71.11095

In [35]:
for key in filteredAlerts.keys():
    if (filteredAlerts[key]['latitude'] == '42.356275' and filteredAlerts[key]['longitude'] == '-71.119852' \
        and filteredAlerts[key]['startTime'] == 56100):
        print(key)
        print(filteredAlerts[key])

idx_1633
{u'latitude': u'42.356275', u'alertType': u'JAM', u'longitude': u'-71.119852', u'startTime': 56100.0, u'endTime': 56160.0}


In [36]:
for key in filteredAlerts.keys():
    if (filteredAlerts[key]['latitude'] == '42.373336' and filteredAlerts[key]['longitude'] == '-71.097731' \
        and filteredAlerts[key]['startTime'] == 77700):
        print(key)
        print(filteredAlerts[key])

idx_3139
{u'latitude': u'42.373336', u'alertType': u'JAM', u'longitude': u'-71.097731', u'startTime': 77700.0, u'endTime': 77760.0}


In [37]:
for key in filteredAlerts.keys():
    if (filteredAlerts[key]['latitude'] == '42.342549263926' and \
        filteredAlerts[key]['longitude'] == '-71.085010881845' \
        and filteredAlerts[key]['startTime'] == 69900):
        print(key)
        print(filteredAlerts[key])

idx_2643
{u'latitude': u'42.342549263926', u'alertType': u'ROAD_CLOSED', u'longitude': u'-71.085010881845', u'startTime': 69900.0, u'endTime': 69960.0}


In [38]:
for key in filteredAlerts.keys():
    if (filteredAlerts[key]['latitude'] == '42.355048' and filteredAlerts[key]['longitude'] == '-71.110335' \
        and filteredAlerts[key]['startTime'] == 77700):
        print(key)
        print(filteredAlerts[key])

idx_3118
{u'latitude': u'42.355048', u'alertType': u'JAM', u'longitude': u'-71.110335', u'startTime': 77700.0, u'endTime': 77760.0}


In [39]:
from __future__ import division

56100/3600, 69900/3600, 77700/3600

(15.583333333333334, 19.416666666666668, 21.583333333333332)